In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import nltk
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/Sorcerer\'s-Stone.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['dursley', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'that', 'they', 'were']
size corpus: 44415
size of unique words: 5601


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('harry', 1325), ('said', 794), ('they', 689), ('that', 686)]
Sample data [131, 466, 241, 495, 388, 8, 1128, 4, 3, 8] ['dursley', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'that', 'they', 'were']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  112.073013306
Nearest to said: alarming, airily, fluff, nails, toads, muffled, -the, during,
Nearest to toward: unknown, scratchy, kiss, above, tokens, mellow, cozy, horny,
Nearest to come: lines, wider, four-legged, weigh, steel, stumbling, nastier, gingerly,
Nearest to something: blackpool, -the, boys, mysteriously, boogers, vain, bundles, standard,
Nearest to where: houses, string, swelling, jabbed, woman, drained, drooling, velvet,
Nearest to back: hurry, grimly, been, frantic, served, countryside, answer, shhh,
Nearest to this: press, snapped, enough, fascinated, eagerly, stones, tripped, guess,
Nearest to little: tackling, unusual, parted, maroon, knowledge, slay, urgent, eerie,
Nearest to again: sitting, forge, bundled, sorcerer, forests, bawling, uncomfortable, wriggling,
Nearest to never: wham, cinemas, blood-red, fields, stopped, chapter, personally, wizard,
Nearest to right: send, presents, helicopters, merrily, occurred, ache, brandish

Average loss at step  52000 :  2.34221208477
Average loss at step  54000 :  2.36303549343
Average loss at step  56000 :  2.32081889188
Average loss at step  58000 :  2.32094636327
Average loss at step  60000 :  2.30534393463
Nearest to said: sharing, acting, liquid, gulpin, outrage, muttered, wildly, edging,
Nearest to toward: scratchy, during, cackle, flung, steps, paracelsus, cozy, shortcut,
Nearest to come: receive, remorse, turkeys, tied, -lessons, definitely, here, train,
Nearest to something: lights, plump, lifted, inferno, chuckle, blackpool, warmer, sprinting,
Nearest to where: perfect, glistening, swelling, relieved, directions, fanatic, rubbed, self-protection,
Nearest to back: hurry, grimly, served, acting, emporium, seeing, tentacles, climb,
Nearest to this: promised, prune, chained, overshadowed, safe, eagerly, umbrella, dittany,
Nearest to little: tackling, cafru, parted, blast, tiptoe, examined, eclairs, relieved,
Nearest to again: drag, nothing, bawling, boomed, forge, 

Average loss at step  102000 :  2.18051435158
Average loss at step  104000 :  2.21609247495
Average loss at step  106000 :  2.18951174439
Average loss at step  108000 :  2.19429281624
Average loss at step  110000 :  2.1843209455
Nearest to said: sharing, liquid, goat, muttered, gulpin, tentacles, wildly, switching,
Nearest to toward: cackle, scratchy, steps, during, pumpkins, flung, unused, petrificus,
Nearest to come: -lessons, tied, receive, child, here, insisted, carelessly, explaining,
Nearest to something: lights, plump, relatives, such, sweaters, lifted, brooms, warmer,
Nearest to where: glistening, fanatic, self-protection, relieved, fuming, rubbed, directions, flies,
Nearest to back: hurry, dusk, emporium, climb, served, linen, holes, bendy,
Nearest to this: prune, dittany, safe, refusing, lamps, barely, should, hags,
Nearest to little: tackling, cafru, blast, tiptoe, examined, horrified, glinting, eerie,
Nearest to again: drag, nothing, bawling, among, earths, boomed, forge, c

Average loss at step  152000 :  2.14512104052
Average loss at step  154000 :  2.17835669488
Average loss at step  156000 :  2.14414642468
Average loss at step  158000 :  2.16853018728
Average loss at step  160000 :  2.15105929178
Nearest to said: bellowed, gulpin, wildly, sharing, fortune-telling, acting, tenpin, panicking,
Nearest to toward: scratchy, cackle, flung, pumpkins, during, nodded, steps, unused,
Nearest to come: -lessons, child, insisted, tied, laws, carelessly, here, receive,
Nearest to something: lights, such, plump, warmer, wished, relatives, struck, lifted,
Nearest to where: fuming, fanatic, glistening, flies, directions, forbid, splashes, rubbed,
Nearest to back: cooked, served, holes, acting, aaaargh, prophet, dusk, climb,
Nearest to this: prune, hags, overshadowed, lamps, chained, should, promised, dittany,
Nearest to little: tackling, horrified, blast, cafru, glinting, tiptoe, demands, wailed,
Nearest to again: drag, bawling, nothing, earths, choice, boomed, forge, 

In [8]:
with open('sorcererstone-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('sorcererstone-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)